### Make boundary conditions for present day experiment

Based on observations from B-SOSE (http://sose.ucsd.edu/)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cmocean
import matplotlib.backends.backend_pdf
import sys
sys.path.append('/home/users/birgal/')
from nemo_python.interpolation import interp_latlon_cf

#### Functions

In [26]:
def prepare_bdy_transect(month=1, bdy_lat=-50):
    # Arguments:
    # month --- to look at
    # bdy_lat --- latitude of boundary slice
    # Returns: WOA18 and B-SOSE salinity and temperatures at the specified boundary latitude

    # Load datasets:
    SOSE_sal  = xr.open_dataset(f'{folder_SOSE}SALT_climatology_m{month:02}.nc').sel(YC=slice(bdy_lat-1, bdy_lat+1))
    SOSE_temp = xr.open_dataset(f'{folder_SOSE}THETA_climatology_m{month:02}.nc').sel(YC=slice(bdy_lat-1, bdy_lat+1))

    # Find strip of points nearest bdy_lat (with edges +/- 1 and fill any zeros with NaNs:
    bdy_ind = np.argmin(np.abs(SOSE_sal.YC.values - bdy_lat))
    SOSE_sal_proc  = xr.where(SOSE_sal.SALT.isel(YC=slice(bdy_ind-1, bdy_ind+2)) ==0, 
                              np.nan, SOSE_sal.isel(YC=slice(bdy_ind-1, bdy_ind+2)))
    SOSE_temp_proc = xr.where(SOSE_temp.THETA.isel(YC=slice(bdy_ind-1, bdy_ind+2)) ==0, 
                              np.nan, SOSE_temp.isel(YC=slice(bdy_ind-1, bdy_ind+2)))

    # And for SOSE, convert longitudes from 0-360 to -180 to 180 for pcolormesh
    SOSE_sal_proc['XC']  = xr.where(SOSE_sal_proc.XC  > 180, SOSE_sal_proc.XC  - 360, SOSE_sal_proc.XC)
    SOSE_temp_proc['XC'] = xr.where(SOSE_temp_proc.XC > 180, SOSE_temp_proc.XC - 360, SOSE_temp_proc.XC)
    
    SOSE_sal_proc  = SOSE_sal_proc.sortby('XC')
    SOSE_temp_proc = SOSE_temp_proc.sortby('XC')
    
    return SOSE_sal_proc, SOSE_temp_proc

In [3]:
def plot_bdy_transect(month, model_S, model_T, SOSE_S, SOSE_T, 
                      Trange=(1,10), Srange=(33.4, 34.6), ylim=(1500,0)):
                         
    fig, ax = plt.subplots(4,1, figsize=(18,12))

    fig.suptitle(f'Month: {month:02}', fontsize=12, fontweight='bold')
                          
    kwargs_T = {'vmin':Trange[0], 'vmax':Trange[1], 'cmap':cmocean.cm.thermal, 'rasterized':True}
    kwargs_S = {'vmin':Srange[0], 'vmax':Srange[1], 'cmap':cmocean.cm.haline, 'rasterized':True}
    
    ax[0].set_title('B-SOSE observations')
    ax[1].set_title('Model boundary condition')
    ax[2].set_title('B-SOSE observations')
    ax[3].set_title('Model boundary condition')
    
    cm1 = ax[0].pcolormesh(SOSE_T.XC, -1*SOSE_T.Z, SOSE_T, **kwargs_T)
    cm2 = ax[1].pcolormesh(SOSE_T.XC, -1*SOSE_T.Z, SOSE_T, **kwargs_T)
    cm3 = ax[2].pcolormesh(SOSE_S.XC, -1*SOSE_S.Z, SOSE_S, **kwargs_S)
    cm4 = ax[3].pcolormesh(SOSE_S.XC, -1*SOSE_S.Z, SOSE_S, **kwargs_S)
    
    CB2 = fig.colorbar(cm2, ax=ax[0:2], fraction=0.03, extend='both', label='Temperature (degC)')
    CB4 = fig.colorbar(cm4, ax=ax[2:], fraction=0.03, extend='both', label='Salinity (g/kg)')
    
    for axis in ax.ravel():
        axis.set_ylabel('Depth (m)')
        axis.invert_yaxis()
        axis.set_ylim(ylim[0],ylim[1])
    ax[3].set_xlabel('Longitude')

    return fig

#### Definitions

In [98]:
# File locations
folder_SOSE = '/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/B-SOSE/climatology/'
folder_BC   = '/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/boundary-conditions/'
folder_NEMO = '/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/bathymetry/'

coordinates_file='/gws/nopw/j04/terrafirma/kaight/NEMO_AIS/coordinates_AIS.nc'
meshmask_file   ='/gws/nopw/j04/terrafirma/birgal/NEMO_AIS/bathymetry/mesh_mask-20231017.nc'

#### Make boundary condition

In [101]:
# Load files:
nemo = xr.open_dataset(coordinates_file).squeeze()
mesh = xr.open_dataset(meshmask_file)

In [104]:
month = 1

# Index of boundary location (Python-based numbering); assumes you want all longitudes
bdy_ind = 452
bdy_lat = nemo.nav_lat.isel(y=452).mean().values

# Load SOSE observational data with a one cell buffer on either side of the boundary latitude
SOSE_S, SOSE_T = prepare_bdy_transect(month=month, bdy_lat=bdy_lat)
SOSE_S_mod     = SOSE_S.rename({'XC':'lon', 'YC':'lat'})
SOSE_T_mod     = SOSE_T.rename({'XC':'lon', 'YC':'lat'})

In [108]:
# Interpolate observations from source to nemo grid along the longitude and latitude grid:
datasets = []
for depth in SOSE_depths:
    source = xr.Dataset({'lon':SOSE_S_mod.lon, 'lat':SOSE_S_mod.lat, 
                         'SALT':SOSE_S_mod.SALT.sel(Z=depth), 'THETA':SOSE_T_mod.THETA.sel(Z=depth)})

    # Interpolate slices of depth levels along lat-lon: returns NaNs for regions of domain not near the boundary
    data_interp = interp_latlon_cf(source, nemo, pster_src=False, periodic_src=True, periodic_nemo=True, method='conservative')

    # Add interpolated layers to dataset, selecting only the slice of values at the specified boundary
    datasets.append(data_interp.isel(y=slice(bdy_ind, bdy_ind+1)))
    
SOSE_horizon_interp = xr.concat(datasets, dim='SOSE_depth')

In [109]:
SOSE_horizon_interp

<xarray.Dataset>
Dimensions:  (SOSE_depth: 52, y: 1, x: 1440)
Dimensions without coordinates: SOSE_depth, y, x
Data variables:
    SALT     (SOSE_depth, y, x) float64 33.59 33.59 33.59 33.59 ... nan nan nan
    THETA    (SOSE_depth, y, x) float64 4.121 4.117 4.115 4.121 ... nan nan nan

In [110]:
SOSE_depths = SOSE_S_mod.Z.values
nemo_depths = mesh.nav_lev.values

In [114]:
print(SOSE_depths.shape)
print(nemo_depths.shape)

(52,)
(121,)


In [224]:
# Compute edges of the z-levels
def vertical_edges(mesh, type='nemo'):

    if type=='nemo':
        z_centres = mesh.gdept_0.isel(time_counter=0, y=0, x=0).values
        dz        = mesh.e3t_0.isel(time_counter=0, y=0, x=0).values
    elif type=='SOSE':
        z_centres = -1*mesh.Z.values 
        dz        = mesh.drF.values
    
    z_top_edge = z_centres - 0.5*dz
    z_bot_edge = z_centres + 0.5*dz
      
    return (z_top_edge, z_centres, z_bot_edge)

In [225]:
nemo_edges = vertical_edges(mesh, type='nemo')
sose_edges = vertical_edges(SOSE_S_mod, type='SOSE')

Alright... I think the method below calculates things correctly in a conservative manner. Now, just need to rewrite it in a more generalized and vectorized way.

In [350]:
cell=9

NEMO_top_edge = nemo_edges[0][cell]
if NEMO_top_edge < 0:
    NEMO_top_edge = 0
NEMO_bot_edge = nemo_edges[2][cell]
print(f'NEMO cell: {NEMO_top_edge:.2f}-{NEMO_bot_edge:.2f}')

Salt = 0
# find the SOSE edges that fall within this depth range:
for zs, zs_centres in enumerate(sose_edges[1]):
    SOSE_top_edge = sose_edges[0][zs]
    SOSE_bot_edge = sose_edges[2][zs]
    
    if (NEMO_top_edge >= SOSE_top_edge) and (NEMO_bot_edge < SOSE_bot_edge):
        # case 1: basic full overlap case
        print('case 1: basic full overlap')
        print(f'SOSE: {SOSE_top_edge:.2f}-{SOSE_bot_edge:.2f}') 
        Salt = (SOSE_S_mod.SALT.isel(lat=0, lon=0)[zs].values)*(NEMO_bot_edge - NEMO_top_edge)
        print('---------')
        
    elif ((NEMO_top_edge >= SOSE_top_edge) and (SOSE_bot_edge < NEMO_bot_edge) and (SOSE_bot_edge > NEMO_top_edge)):
        print('case 2: multiple cells overlap (this cell is partially shallower)')
        print(f'SOSE: {SOSE_top_edge:.2f}-{SOSE_bot_edge:.2f}') 
        Salt = Salt + (SOSE_bot_edge - NEMO_top_edge)*SOSE_S_mod.SALT.isel(lat=0, lon=0)[zs].values
        
    elif ((NEMO_top_edge < SOSE_top_edge) and (SOSE_bot_edge > NEMO_bot_edge) and (SOSE_top_edge < NEMO_bot_edge)):
        print('case 2: multiple cells overlap  (this cell is partially deeper)')
        print(f'SOSE: {SOSE_top_edge:.2f}-{SOSE_bot_edge:.2f}') 
        Salt = Salt + (NEMO_bot_edge - SOSE_top_edge)*SOSE_S_mod.SALT.isel(lat=0, lon=0)[zs].values
        
print(f'Salt: {Salt/(NEMO_bot_edge - NEMO_top_edge):.5f}')

NEMO cell: 12.95-15.29
case 2: multiple cells overlap (this cell is partially shallower)
SOSE: 9.20-15.10
case 2: multiple cells overlap  (this cell is partially deeper)
SOSE: 15.10-22.00
Salt: 33.78824


In [290]:
top=10
for z, z_centres in enumerate(nemo_edges[1][1:top]):
    top_edge = nemo_edges[0][z]
    if top_edge < 0:
        top_edge = 0
    bot_edge = nemo_edges[2][z]

    # find the SOSE edges that fall within this depth range:
    for zs, zs_centres in enumerate(sose_edges[1][0:top]):
        if (top_edge <= sose_edges[0][zs]) and (bot_edge < sose_edges[2][zs]) and (sose_edges[0][zs] < bot_edge):
            # case 1: basic overlap
            print('case 1')
            print(f'NEMO: {top_edge:.2f}-{bot_edge:.2f}') 
            print(f'SOSE: {sose_edges[0][zs]:.2f}-{sose_edges[2][zs]:.2f}') 
            print(f'z: {z}, {SOSE_S_mod.SALT.isel(lat=0, lon=0)[zs].values:.2f}')
            print('---------')
            
        if (sose_edges[0][zs] < bot_edge) and (sose_edges[2][zs] > bot_edge):
            print('case 2')
            print(f'NEMO: {top_edge:.2f}-{bot_edge:.2f}') 
            print(f'SOSE: {sose_edges[0][zs]:.2f}-{sose_edges[2][zs]:.2f}') 
            print(f'z: {z}, {SOSE_S_mod.SALT.isel(lat=0, lon=0)[zs].values:.2f}')
            print('---------')

    # #if multiple SOSE cells contribute to the NEMO cell, make sure to weight by thickness:
    # salt_weighted = SOSE_S_mod.SALT.isel(lat=0, lon=0, Z=zs)*(bot_edge-top_edge) 
    # salt_weighted = SOSE_S_mod.SALT.isel(lat=0, lon=0, Z=zs)*()
    # SOSE_in_NEMO = 
        

    # for source cells that fall within the edges of the nemo cell, calculate the average:
    # SOSE_S_mod.SALT.isel(z=())
    # sose_edges

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
# Interpolate along the vertical: Needs to be conservative to keep the fluxes the same
data_interp = interp_depth(source, nemo, method='conservative') # returns NaNs for regions of domain not near the boundary

# define the target values in density
theta_target = np.linspace(0,3, 20)

# and transform
phi_transformed_cons = grid.transform(ds.phi,
                                      'Z',
                                      theta_target,
                                      method='conservative',
                                      target_data=ds.theta_outer)
phi_transformed_cons

# SOSE_vertic_interp = xr.concat(datasets, dim='depth')

In [ ]:

def interp_depth(source, nemo):
    import cf

    
    
    source.depth
    nemo.nav_lev
    
    interp = xr.Dataset()
    for var, data_cf0 in zip(source, data_cf):
        data_interp = data_cf0.regrids(regrid_operator, src_axes=src_axes).array
        data_interp = xr.DataArray(data_interp, dims=['z', 'x'])
        interp = interp.assign({var:data_interp})     

    return

In [ ]:
fig1 = {}
for month in range(1,13):
    WOA_S, WOA_T, SOSE_S, SOSE_T = prepare_bdy_transect(month=month, bdy_lat=-50)
    fig1[f'm{month:02}']         = plot_bdy_transect(month, model_S, model_T, SOSE_S, SOSE_T, 
                                                     Trange=(0,10), Srange=(33.6, 34.8), ylim=(1500,0))

# pdf1 = matplotlib.backends.backend_pdf.PdfPages(f"{folder_terra}WOA_SOSE_boundary_transect_1500m.pdf")
# for month in range(1,13):
#     pdf1.savefig(fig1[f'm{month:02}'])
# pdf1.close()